<h1>Face Detection</h1>

In [ ]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

face_detection = mp_face_detection.FaceDetection(
    model_selection=1,  
    min_detection_confidence=0.5
)

image = cv2.imread('./Data/1 (921).jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = face_detection.process(image_rgb)

if results.detections:
    for i, detection in enumerate(results.detections):
        bboxC = detection.location_data.relative_bounding_box
        ih, iw, _ = image.shape
        
        x = int(bboxC.xmin * iw)
        y = int(bboxC.ymin * ih)
        w = int(bboxC.width * iw)
        h = int(bboxC.height * ih)
        
        face_img = image[y:y+h, x:x+w]
        cv2.imwrite(f'face_{i}.jpg', face_img)